In [2]:
import scipy.sparse as sp
import numpy as np
import pandas as pd
from collections import Counter

In [89]:
ncbi_gtdb = pd.read_table('/home/miaocj/docker_dir/data/cami_download/setup/all_od',sep ='\t',header=None,dtype=str)
df = ncbi_gtdb.iloc[:,[0,1,3]]
df.columns = ['species','id','first_id']
df.loc[df["id"].isna(), "id"] = df.loc[df["id"].isna(), "first_id"]
df.iloc[:,[0,1]].to_csv('/home/miaocj/docker_dir/data/cami_download/setup/gtdb_all_id',sep='\t',header=None,index=None)

/tmp/ipykernel_496663/2570102267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df["id"].isna(), "id"] = df.loc[df["id"].isna(), "first_id"]


In [120]:
gtdb = pd.read_table('/home/miaocj/docker_dir/data/cami_download/setup/cami_gtdb.info',sep ='\t',header=None,dtype=str)
ncbi = pd.read_table('/home/miaocj/docker_dir/data/cami_download/setup/cami_ncbi1.info',sep ='\t',header=None,dtype=str)
ncbi.columns = ['id','species']
gtdb.columns = ['id','lineage']
ncbi = ncbi.drop_duplicates()
gtdb = gtdb.drop_duplicates()
df = df.drop_duplicates()
gtdb_id_sp = pd.merge(df, gtdb, on="id")
final_output = pd.merge(ncbi, gtdb_id_sp, on="species")
final_output.to_csv('/home/miaocj/docker_dir/data/cami_download/setup/ncbi2gtdb.csv',sep ='\t',header=None,index=None)

In [172]:
abundance = pd.read_table('/home/miaocj/docker_dir/data/cami_download/setup/abundance0.tsv',sep ='\t',header=None)
metadata =  pd.read_table('/home/miaocj/docker_dir/data/cami_download/setup/metadata.tsv',sep ='\t',dtype=str)
read_map =  pd.read_table('/home/miaocj/docker_dir/data/cami_download/2018.01.23_11.53.11_sample_0/reads/reads_mapping.tsv',sep ='\t',dtype=str)
ncbi2gtdb = final_output.iloc[:,[0,1,2,4]]
ncbi2gtdb.columns = ['NCBI_ID','species','GTDB_id','lineage']
abundance.columns = ['genome_ID','abundance']
sample0_ab = abundance[abundance.abundance != 0]
merge1 = pd.merge(sample0_ab,metadata,on='genome_ID')
merge2 = pd.merge(merge1,ncbi2gtdb,on='NCBI_ID',how='left')
merge2.to_csv('/home/miaocj/docker_dir/data/cami_download/2018.01.23_11.53.11_sample_0/reads/abundance_lineage.tsv',sep='\t',index=None)

In [187]:
merge2[merge2.GTDB_id.isna()]

,genome_ID,abundance,OTU,NCBI_ID,novelty_category,species,GTDB_id,lineage
24,OTU_97.45053.1,0.011318,724,730,new_strain,NaN,NaN,NaN
26,OTU_97.6577.0,1.000000,1485,1029718,new_strain,NaN,NaN,NaN
65,OTU_97.45058.0,1.000000,724,730,new_strain,NaN,NaN,NaN


In [158]:
pd.merge(sample0_ab,metadata,on='')

,genome_ID,abundance,OTU,NCBI_ID,novelty_category
0,OTU_97.14013.0,4.000000,506,520,new_strain
1,OTU_97.44483.0,20.000000,541000,1121335,new_strain
2,OTU_97.18736.0,3.000000,838,246198,new_strain
3,OTU_97.4094.1,0.993374,1485,1231072,new_strain
4,OTU_97.4094.0,0.006626,1485,1520,new_strain
...,...,...,...,...,...
64,OTU_97.3641.0,1.000000,1485,441772,new_strain
65,OTU_97.45058.0,1.000000,724,730,new_strain
66,OTU_97.584.0,11.000000,506,520,new_strain
67,OTU_97.37167.0,2.000000,171551,1642646,new_strain


In [82]:
df.loc[df["id"].isna(), "id"] = df.loc[df["id"].isna(), "first_id"]

In [86]:
df

,species,id,first_id
0,Streptococcus pyogenes,2.134891e+09,1.804775e+09
1,Campylobacter coli,3.397761e+08,3.397761e+08
2,Eubacterium limosum,7.064186e+06,4.398649e+07
3,Mannheimia varigena,1.185383e+09,1.981471e+09
4,Burkholderia pseudomallei,6.622622e+08,6.622622e+08
...,...,...,...
1052,Filifactor alocis,3.760064e+08,3.513520e+07
1053,Lachnoclostridium sp. YL32,9.454648e+08,9.454648e+08
1054,Neisseria meningitidis,7.034828e+08,1.740130e+09
1055,Neisseria gonorrhoeae,2.068206e+09,1.740130e+09


In [87]:
df.loc[df["first_id"].isna(), :]

,species,id,first_id
61,Myroides profundi,NaN,NaN
70,[Haemophilus] ducreyi,NaN,NaN
105,Lawsonia intracellularis,NaN,NaN
144,[Clostridium] cellulosi,NaN,NaN
150,[Mannheimia] succiniciproducens,NaN,NaN
256,[Haemophilus] ducreyi,NaN,NaN
275,[Haemophilus] ducreyi,NaN,NaN
292,[Haemophilus] ducreyi,NaN,NaN
310,[Haemophilus] ducreyi,NaN,NaN
368,Schaalia meyeri,NaN,NaN


In [78]:
df.iloc[:,[0,1]].to_csv('/home/miaocj/docker_dir/data/cami_download/setup/gtdb_all_id',sep='\t')

In [22]:
easy_info_path = '~/docker_dir/kNN-overlap-finder/data/regional_reads/yeast/chr4_1M/real_cyclone_easy/read_info.tsv'
easy_info_df = pd.read_table(easy_info_path)
old_info_path = '~/docker_dir/kNN-overlap-finder/data/regional_reads/yeast/chr4_1M/real_cyclone_old/read_info.tsv'
old_info_df = pd.read_table(old_info_path)

In [5]:
import re
cami_id = []
with open('/home/miaocj/docker_dir/data/cami_download/setup/genome_to_id.tsv','rt') as f:
    for lines in f:
        match = re.search(r'GCA_(\d+)\.',lines)
        id = match.group(1)
        cami_id.append(id)

In [17]:
from Bio import Entrez

# 设置邮箱（NCBI 要求）
Entrez.email = "your_email@example.com"

# 获取分类学信息
def get_taxonomy_info(tax_id):
    handle = Entrez.efetch(db="taxonomy", id=str(tax_id))
    record = Entrez.read(handle)
    handle.close()
    return record[0]

# 示例：获取 Taxonomy ID 9606（人类）的分类学信息
tax_id = 9606
tax_info = get_taxonomy_info(tax_id)

In [19]:
tax_info.keys()

dict_keys(['TaxId', 'ScientificName', 'OtherNames', 'ParentTaxId', 'Rank', 'Division', 'GeneticCode', 'MitoGeneticCode', 'Lineage', 'LineageEx', 'CreateDate', 'UpdateDate', 'PubDate'])

In [54]:
df = pd.read_csv('/home/miaocj/docker_dir/data/cami_download/2018.01.23_11.53.11_sample_0/reads/reads_mapping.tsv',sep='\t')

In [59]:
df

,#anonymous_read_id,genome_id,tax_id,read_id
0,S0R0,OTU_97.23934.0,506,HE965807.1-4925
1,S0R1,OTU_97.44483.0,541000,CP004044.1-66968
2,S0R10,OTU_97.40119.0,506,HE965805.1-25838
3,S0R100,OTU_97.7000.0,506,CP016440.1-63351
4,S0R1000,OTU_97.68.0,841,CP003040.1-15640
...,...,...,...,...
1666302,S0R999995,OTU_97.4796.0,415002,CP001998.1-31505
1666303,S0R999996,OTU_97.6890.0,31977,CP009240.1-1646
1666304,S0R999997,OTU_97.498.0,506,CP010840.1-52248
1666305,S0R999998,OTU_97.23934.0,506,HE965807.1-22


In [60]:
len(set(df.genome_id.tolist()))

69

In [61]:
len(set(df.tax_id.tolist()))

17

In [ ]:
x = []
for di in tax_info['LineageEx']:
    x.append(di['Rank'])
    if di['Rank'] == 'species':
        print(di)

In [51]:
x

['no rank',
 'superkingdom',
 'clade',
 'kingdom',
 'clade',
 'clade',
 'clade',
 'phylum',
 'subphylum',
 'clade',
 'clade',
 'clade',
 'clade',
 'superclass',
 'clade',
 'clade',
 'clade',
 'class',
 'clade',
 'clade',
 'clade',
 'superorder',
 'order',
 'suborder',
 'infraorder',
 'parvorder',
 'superfamily',
 'family',
 'subfamily',
 'genus']

In [14]:
len(cami_id)

1057

In [15]:
import os 
pass_id = []
for id in cami_id:
    path1 = '/home/miaocj/docker_dir/data/GTDB_download/release220/skani/database/GCA/' + str(id[:3]) + '/' + str(id[3:6]) + '/' + str(id[6:])
    path2 = '/home/miaocj/docker_dir/data/GTDB_download/release220/skani/database/GCF/' + str(id[:3]) + '/' + str(id[3:6]) + '/' + str(id[6:])
    if  os.path.exists(path1) or os.path.exists(path2):
        pass_id.append(id)


In [16]:
len(pass_id)

225

In [13]:
len(noid)

832

In [31]:
easy_info_df.sort_values(by="read_name").iloc[:10,:]

,read_name,read_length,reference_strand,reference_start,reference_end
0,TB2000DC54-202404161449270_20240416162943_0000...,2851,-,614105,616955
1,TB2000DC54-202404161449270_20240416162943_0000...,427,-,1471587,1472023
2,TB2000DC54-202404161449270_20240416162943_0000...,1580,-,1271751,1273330
3,TB2000DC54-202404161449270_20240416162943_0000...,2118,+,1027017,1029190
4,TB2000DC54-202404161449270_20240416162943_0001...,694,+,1016604,1017297
5,TB2000DC54-202404161449270_20240416162943_0001...,488,-,965036,965516
6,TB2000DC54-202404161449270_20240416162943_0001...,129,+,87971,88093
7,TB2000DC54-202404161449270_20240416162943_0001...,917,-,1037653,1038563
8,TB2000DC54-202404161449270_20240416162943_0002...,4182,-,1130168,1134355
9,TB2000DC54-202404161449270_20240416162943_0002...,945,-,1498483,1499430


In [32]:
old_info_df.sort_values(by="read_name").iloc[:10,:]

,read_name,read_length,reference_strand,reference_start,reference_end
4659,TB2000DC54-202404161449270_20240416162943_0000...,2851,-,614108,616950
4177,TB2000DC54-202404161449270_20240416162943_0000...,427,-,1471596,1472017
482,TB2000DC54-202404161449270_20240416162943_0000...,1580,-,1271761,1273326
6478,TB2000DC54-202404161449270_20240416162943_0000...,2118,+,1027024,1029183
5746,TB2000DC54-202404161449270_20240416162943_0001...,694,+,1016647,1017296
4045,TB2000DC54-202404161449270_20240416162943_0001...,488,-,965041,965504
5070,TB2000DC54-202404161449270_20240416162943_0001...,129,+,87982,88086
35,TB2000DC54-202404161449270_20240416162943_0001...,917,-,1037661,1038562
3379,TB2000DC54-202404161449270_20240416162943_0002...,4182,-,1130169,1134346
4590,TB2000DC54-202404161449270_20240416162943_0002...,945,-,1498489,1499423


In [130]:
merged_inner = pd.merge(easy_info_df, old_info_df, on='read_name', how='left')

In [131]:
merged_inner['minus'] = merged_inner['reference_start_x']- merged_inner['reference_start_y']

In [8]:
easy_feature_matrix = sp.load_npz('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/yeast/chr4_1M/real_cyclone_easy/kmer_k16/feature_matrix.npz')
old_feature_matrix = sp.load_npz('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/yeast/chr4_1M/real_cyclone_old/kmer_k16/feature_matrix.npz')

In [5]:
easy_feature_matrix.shape

(14386, 545203)

In [9]:
old_feature_matrix.shape

(13092, 525405)

In [16]:
easymeta_df = pd.read_table('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/yeast/chr4_1M/real_cyclone_easy/kmer_k16/metadata.tsv.gz')
oldmeta_df = pd.read_table('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/yeast/chr4_1M/real_cyclone_old/kmer_k16/metadata.tsv.gz')

In [35]:
data = np.load('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/yeast/chr4_1M/real_cyclone/kmer_k16/Exact_Cosine_None_IDF_nbr_matrix.npz') 
nbr_indices = data['arr_0']

In [55]:
for i1, row in enumerate(nbr_indices):
    row = row[(row >= 0) & (row != i1)]
    for neighbor_order, i2 in enumerate(row[1:2]):
        print((i1,i2, {"neighbor_order": neighbor_order}))

(0, 904, {'neighbor_order': 0})
(1, 905, {'neighbor_order': 0})
(2, 4903, {'neighbor_order': 0})
(3, 4902, {'neighbor_order': 0})
(4, 6781, {'neighbor_order': 0})
(5, 6780, {'neighbor_order': 0})
(6, 637, {'neighbor_order': 0})
(7, 636, {'neighbor_order': 0})
(8, 8722, {'neighbor_order': 0})
(9, 8723, {'neighbor_order': 0})
(10, 8436, {'neighbor_order': 0})
(11, 8437, {'neighbor_order': 0})
(12, 1676, {'neighbor_order': 0})
(13, 1677, {'neighbor_order': 0})
(14, 8275, {'neighbor_order': 0})
(15, 8274, {'neighbor_order': 0})
(16, 4785, {'neighbor_order': 0})
(17, 4784, {'neighbor_order': 0})
(18, 9090, {'neighbor_order': 0})
(19, 9091, {'neighbor_order': 0})
(20, 4066, {'neighbor_order': 0})
(21, 4067, {'neighbor_order': 0})
(22, 8124, {'neighbor_order': 0})
(23, 8125, {'neighbor_order': 0})
(24, 6241, {'neighbor_order': 0})
(25, 6240, {'neighbor_order': 0})
(26, 4475, {'neighbor_order': 0})
(27, 4474, {'neighbor_order': 0})
(28, 3412, {'neighbor_order': 0})
(29, 3413, {'neighbor_order'

In [34]:
easymeta_df.sort_values(by='read_name').iloc[10:20,:]

,read_id,read_name,read_orientation,read_length,reference_strand,reference_start,reference_end
10,10,TB2000DC54-202404161449270_20240416162943_0001...,+,488,-,965036,965516
11,11,TB2000DC54-202404161449270_20240416162943_0001...,-,488,+,965036,965516
12,12,TB2000DC54-202404161449270_20240416162943_0001...,+,129,+,87971,88093
13,13,TB2000DC54-202404161449270_20240416162943_0001...,-,129,-,87971,88093
15,15,TB2000DC54-202404161449270_20240416162943_0001...,-,917,+,1037653,1038563
14,14,TB2000DC54-202404161449270_20240416162943_0001...,+,917,-,1037653,1038563
16,16,TB2000DC54-202404161449270_20240416162943_0002...,+,4182,-,1130168,1134355
17,17,TB2000DC54-202404161449270_20240416162943_0002...,-,4182,+,1130168,1134355
18,18,TB2000DC54-202404161449270_20240416162943_0002...,+,945,-,1498483,1499430
19,19,TB2000DC54-202404161449270_20240416162943_0002...,-,945,+,1498483,1499430


In [33]:
oldmeta_df.sort_values(by='read_name').iloc[10:20,:]

,read_id,read_name,read_orientation,read_length,reference_strand,reference_start,reference_end
8091,8091,TB2000DC54-202404161449270_20240416162943_0001...,-,488,+,965041,965504
8090,8090,TB2000DC54-202404161449270_20240416162943_0001...,+,488,-,965041,965504
10140,10140,TB2000DC54-202404161449270_20240416162943_0001...,+,129,+,87982,88086
10141,10141,TB2000DC54-202404161449270_20240416162943_0001...,-,129,-,87982,88086
70,70,TB2000DC54-202404161449270_20240416162943_0001...,+,917,-,1037661,1038562
71,71,TB2000DC54-202404161449270_20240416162943_0001...,-,917,+,1037661,1038562
6759,6759,TB2000DC54-202404161449270_20240416162943_0002...,-,4182,+,1130169,1134346
6758,6758,TB2000DC54-202404161449270_20240416162943_0002...,+,4182,-,1130169,1134346
9180,9180,TB2000DC54-202404161449270_20240416162943_0002...,+,945,-,1498489,1499423
9181,9181,TB2000DC54-202404161449270_20240416162943_0002...,-,945,+,1498489,1499423


In [47]:
info_path = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/CHM13/HLA/pbsim_ONT_93_10k/read_info.tsv.gz'
meta_df = pd.read_table(info_path)
import json,gzip
json_path = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_10k/kmer_k16/read_features.json.gz'
with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
read_features = {i: read_features[i] for i in meta_df.index} 

In [53]:
json_path = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_10k/kmer_k16/read_features.json.gz'
with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index} 

In [ ]:
meta_df = pd.read_table('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_10k/kmer_k16/metadata.tsv.gz').reset_index()
print(meta_df.shape)
print(max(meta_df.index))

In [56]:
nbr_indices = np.load('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/CHM13/HLA/pbsim_ONT_93_10k/kmer_k16/Exact_Euclidean_None_TF_nbr_matrix.npz')['arr_0']
read_ids = np.array(list(read_features))

In [ ]:
import pickle
with open( , 'rb') as f:
    loaded_list = pickle.load(f)

In [102]:
import pickle
with open('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/CHM13/HLA/pbsim_ONT_93_10k/kmer_k16/Exact_Cosine_None_TF_overlap_sizes.pkl', 'rb') as f:
    loaded_list2 = pickle.load(f)

In [ ]:
len(loaded_list2[0])

In [ ]:
loaded_list2[0].count(0)

In [ ]:
loaded_list[0].count(0)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# 将嵌套列表转换为长格式的 DataFrame
data = []
for i, sublist in enumerate(loaded_list):
    for value in sublist:
        data.append({'Group': f'{i+1}', 'Value': value})
df = pd.DataFrame(data)

# 使用 seaborn 绘制箱型图
plt.figure(figsize=(8, 6),dpi=300)  # 设置图像大小
sns.boxplot(x='Group', y='Value', data=df)

# 添加标题和标签
plt.xlabel('Neighbor Rank')
plt.ylabel('Overlap Size')

# 显示图像
plt.show()

In [ ]:
import sys
import networkx as nx

sys.path.append('../scripts')
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges,get_neighbor_overlap_bases

meta_df = pd.read_table('/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/CHM13/HLA/pbsim_ONT_93_10k/read_info.tsv.gz').reset_index()

def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

In [ ]:
nbr_indices.shape

In [ ]:
read_ids.shape

In [ ]:
graph = OverlapGraph.from_neighbor_indices(
neighbor_indices=nbr_indices,
n_neighbors=20,
read_ids=read_ids,
require_mutual_neighbors=False,)

In [ ]:
nbr['arr_0'].shape

In [ ]:
len(read_features[5])

In [20]:
with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)

In [ ]:
len(read_features)

In [ ]:
meta_df.index

In [17]:
feature_matrix2 = sp.load_npz('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_98_30k/kmer_k16/feature_matrix.npz')[meta_df.index, :]

In [ ]:
feature_matrix = sp.load_npz('/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_98_30k/kmer_k16/feature_matrix.npz')

In [ ]:
feature_matrix.shape

In [ ]:
len(read_features)

In [19]:
a = [0,0,0,0,1,1,1]
b = [0,0,0,0,1,1,0]
c = [1,1,1,1,1,1,1]
d = [0,0,1,1,1,1,0]

In [11]:
def normalize(a):
    return [i/sum(a) for i in a]

In [ ]:
print(euclidean(normalize(a),normalize(b)))
print(euclidean(normalize(c),normalize(b)))
print(cosine(normalize(a),normalize(b)))
print(cosine(normalize(c),normalize(b)))

In [ ]:
print(euclidean(a,b))
print(euclidean(c,b))
print(euclidean(d,b))
print(cosine(a,b))
print(cosine(c,b))

In [23]:
import pysam
import pandas as pd
samfile = pysam.AlignmentFile("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam", "rb")

In [27]:
def get_bam_stat_df(bam_file):
    bam = pysam.AlignmentFile(bam_file, "rb")
    df_rows = []
    for i, seg in enumerate(bam.fetch()):
        if seg.reference_name == 'chr1_MATERNAL':
            if seg.is_reverse:
                strand = "-"  # 反向链
            else:
                strand = "+"  # 正向链
            if seg.has_tag("NM"):
                mismatch =  seg.get_tag("NM")
            else:
                mismatch = 0 
            row = dict(
                segment_id=i,
                reference_name=seg.reference_name,
                reference_start=seg.reference_start,
                reference_end=seg.reference_end,
                
                mismatch=mismatch,
                query_name=seg.query_name,
                query_alignment_start=seg.query_alignment_start,
                query_length=seg.query_length,
                query_alignment_length=seg.query_alignment_length,
                mapping_quality=seg.mapping_quality,
            )
            df_rows.append(row)
    df = pd.DataFrame(df_rows)
    return df

In [28]:
df = get_bam_stat_df("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam")

In [ ]:
df

In [ ]:
##percentage 为reference上的长度占整个querylength的比例
df['percentage'] =(df.reference_end - df.reference_start)/df.query_length
df2 = df.groupby('query_name')['percentage'].max().reset_index()
bad_query = df2[(df2.percentage<0.9) | (df2.percentage>1.2)]['query_name'].tolist()
bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_align[bad_query_align.loc[:,0]>0]

In [ ]:
##percentage 为比对上的长度占整个querylength的比例
df['percentage'] =df['query_alignment_length']/df['query_length']
max_indices = df.groupby('query_name')['percentage'].idxmax()
result = df.loc[max_indices]

df2 = df.groupby('query_name')['percentage'].max().reset_index()
bad_query = df2[df2.percentage<0.9]['query_name'].tolist()
meta_df = df[ ~df.query_name.isin(bad_query)]

bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_align[bad_query_align.loc[:,0]>0]

In [ ]:
len(set(result['query_name'].tolist()))

In [100]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from snakemake_stub import *
from dataclasses import dataclass, field
import collections
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem
import csv
import sys
from intervaltree import Interval, IntervalTree
from collections import defaultdict
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/scripts")

from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges

In [ ]:
def get_bam_stat_df(bam_file):
    bam = pysam.AlignmentFile(bam_file, "rb")
    df_rows = []
    for i, seg in enumerate(bam.fetch()):
        if (seg.reference_name == 'chr6_MATERNAL') and (seg.reference_start > 28476949) and (seg.reference_end < 34231258):
            if seg.is_reverse:
                strand = "-"  # 反向链
            else:
                strand = "+"  # 正向链
            if seg.has_tag("NM"):
                mismatch =  seg.get_tag("NM")
            else:
                mismatch = 0 
            row = dict(
                segment_id=i,
                reference_name=seg.reference_name,
                reference_start=seg.reference_start,
                reference_end=seg.reference_end,
                reference_strand = strand,
                mismatch=mismatch,
                query_name=seg.query_name,
                query_alignment_start=seg.query_alignment_start,
                query_length=seg.query_length,
                query_alignment_length=seg.query_alignment_length,
                mapping_quality=seg.mapping_quality,
            )
            df_rows.append(row)
    df = pd.DataFrame(df_rows)
    return df



df = get_bam_stat_df("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam")
df['percentage'] =df['query_alignment_length']/df['query_length']
max_indices = df.groupby('query_name')['percentage'].idxmax()
result = df.loc[max_indices]
meta_df = result[result.percentage>0.9]
all_reads_num = len(result['query_name'].tolist())

bad_query = result[result.percentage<=0.9].tolist()
bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_num = bad_query_align.shape[0]
bad_in_all = bad_query_num/all_reads_num
lines_number = bad_query_align[bad_query_align.loc[:,0]>1].shape[0]
per = lines_number/bad_query_num
print('There are %d(%f) reads do not have >90% percentage and %d(%f) of them have more than one alignment.'%(bad_query_num,bad_in_all,lines_number,per))
  
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

fig, ax = plt.subplots(figsize=(8, 6))

# 调用 plot_read_graph 函数
plot_read_graph(
    ax=ax,
    metadata=meta_df,        # 替换为你的 metadata
    query_graph=reference_graph,  # 替换为你的 reference_graph
    layout_method="sfdp",     # 布局方法
    node_size=3,              # 节点大小
    seed=43                   # 随机种子
)

fig.savefig("output_graph.png", dpi=300, bbox_inches="tight")
  

In [ ]:

def build_interval_trees(read_dict):
    """
    将每条 read (保留所有比对位点) 根据 reference_name 构建 IntervalTree。
    返回: { ref_name: IntervalTree(...) }
    """
    interval_trees = defaultdict(IntervalTree)
    
    for rname, alignments in read_dict.items():
        for info in alignments:
            ref = info["ref_name"]
            start = info["start"]
            end = info["end"]
            
            # Interval 的 data 存储 read_name；后续再从 read_dict 获取其他信息
            interval_trees[ref].add(Interval(start, end, data=rname))
        
    return interval_trees

In [ ]:
len(set(df.loc[:,'query_name'].tolist()))

In [ ]:
len(set(df[df.percentage > 0.9].loc[:,'query_name'].tolist()))

In [5]:
neighbors = np.load('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/metagenome/bacteria/pbsim_ONT_95_20k/kmer_16/MinHash_Jaccard_None_None_nbr_matrix.npz')

In [11]:
nbr = neighbors['arr_0']

In [ ]:
Counter(nbr[:,0].tolist())